In [1]:
import sys
sys.path.insert(0, './CPPotify/source/py')
from CPPotify import CPPotify

from keys import (USER, PASS, ACCOUNT, CLIENT_ID, CLIENT_SECRET,
                  REDIRECT_URI, STATE, SCOPE, SHOW_DIALOG,
                  ID, SECRET)

import snowflake.connector
import pandas as pd
import boto3
from datetime import datetime
from io import StringIO

bucket = 'spotifyrecsystem'

cpp = CPPotify(CLIENT_ID, CLIENT_SECRET)

In [4]:
def checkpoint():
    ctx = snowflake.connector.connect(
        user = USER,
        password = PASS,
        account = ACCOUNT,print(e)
        warehouse = 'COMPUTE_WH',
        database = 'SPOTIFY_DB'
    )
    cs = ctx.cursor()

    if len(playlist_id_list) != 0:client('connect')
        if len(playlist_id_list) >= 16384:
            for x in range(math.ceil(len(playlist_id_list)/16384)):
                temp = playlist_id_list[x * 16384 : (x * 16384) + 16384]
                playlist_id_tuple = [[str(x)] for x in temp]
                try:
                    query_string = "INSERT INTO PUBLIC.PLAYLIST_IDS(ID) VALUES (%s)"
                    cs.executemany(query_string, playlist_id_tuple)
                except Exception as e:
                    print(e)
        else:
            playlist_id_tuple = [[str(x)] for x in playlist_id_list]
            try:
                query_string = "INSERT INTO PUBLIC.PLAYLIST_IDS(ID) VALUES (%s)"
                cs.executemany(query_string, playlist_id_tuple)
            except Exception as e:
                print(e)

    if len(album_id_list) != 0:
        if len(album_id_list) >= 16384:
            for x in range(math.ceil(len(album_id_list)/16384)):
                temp = album_id_list[x * 16384 : (x * 16384) + 16384]
                album_id_tuple = [[str(x)] for x in temp]
                try:
                    query_string = "INSERT INTO PUBLIC.ALBUM_IDS(ID) VALUES (%s)"
                    cs.executemany(query_string, album_id_tuple)
                except Exception as e:
                    print(e)
        else:
            album_id_tuple = [[str(x)] for x in album_id_list]
            try:
                query_string = "INSERT INTO PUBLIC.ALBUM_IDS(ID) VALUES (%s);"
                cs.executemany(query_string, album_id_tuple)
            except Exception as e:
                print(e)

    if len(artist_id_list) != 0:
        if len(artist_id_list) >= 16384:
            for x in range(math.ceil(len(artist_id_list)/16384)):
                temp = artist_id_list[x * 16384 : (x * 16384) + 16384]
                artist_id_tuple = [[str(x)] for x in temp]
                try:
                    query_string = "INSERT INTO PUBLIC.ARTIST_IDS(ID) VALUES (%s)"
                    cs.executemany(query_string, artist_id_tuple)
                except Exception as e:
                    print(e)
        else:
            artist_id_tuple = [[str(x)] for x in artist_id_list]
            try:
                query_string = "INSERT INTO PUBLIC.ARTIST_IDS(ID) VALUES (%s);"
                cs.executemany(query_string, artist_id_tuple)
            except Exception as e:
                print(e)

    if len(track_id_list) != 0:
        if len(track_id_list) >= 16384:
            for x in range(math.ceil(len(track_id_list)/16384)):
                temp = track_id_list[x * 16384 : (x * 16384) + 16384]
                track_id_tuple = [[str(x)] for x in temp]
                try:
                    query_string = "INSERT INTO PUBLIC.TRACK_IDS(ID) VALUES (%s)"
                    cs.executemany(query_string, track_id_tuple)
                except Exception as e:
                    print(e)
        else:
            track_id_tuple = [[str(x)] for x in track_id_list]
            try:
                query_string = "INSERT INTO PUBLIC.TRACK_IDS(ID) VALUES (%s);"
                cs.executemany(query_string, track_id_tuple)
            except Exception as e:
                print(e)

    ctx.close()
    cs.close()

In [26]:
def audio_features_only_checkpoint():
    ctx = snowflake.connector.connect(
        user = USER,
        password = PASS,
        account = ACCOUNT,
        warehouse = 'COMPUTE_WH',
        database = 'SPOTIFY_DB'
    )
    cs = ctx.cursor()

    if len(audio_features_df) != 0:
        start = (len(audio_features_df)//10000 - 1) * 10000
        audio_features_tuple = list(audio_features_df.iloc[start : start + 10000].itertuples(index=False, name=None))
        try:
            query_string = "INSERT INTO PUBLIC.AUDIO_FEATURES_ONLY(DANCEABILITY, ENERGY, KEY, LOUDNESS, MODE, SPEECHINESS, ACOUSTICNESS, INSTRUMENTALNESS, LIVENESS, VALENCE, TEMPO, TYPE, ID, URI, TRACK_HREF, ANALYSIS_URL, DURATION_MS, TIME_SIGNATURE) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);"
            cs.executemany(query_string, audio_features_tuple)
        except Exception as e:
            print(e)
            
    ctx.close()
    cs.close()

In [10]:
client = boto3.client(
        's3',
        aws_access_key_id = ID, 
        aws_secret_access_key = SECRET,
        region_name = 'us-west-1'
)

resource = boto3.resource(
        's3',
        aws_access_key_id = ID, 
        aws_secret_access_key = SECRET,
        region_name = 'us-west-1'
    )

for obj in client.list_objects(Bucket='spotifyrecsystem')['Contents']:
    if obj['Key'] == 'artists/artists_data_cache.csv':
        resource.Object('spotifyrecsystem', obj['Key']).delete()

In [27]:
def audio_features_full_checkpoint():
    ctx = snowflake.connector.connect(
        user = USER,
        password = PASS,
        account = ACCOUNT,
        warehouse = 'COMPUTE_WH',
        database = 'SPOTIFY_DB'
    )
    cs = ctx.cursor()

    if len(audio_features_full_df) != 0:
        start = (len(audio_features_full_df)//10000 - 1) * 10000
        audio_features_tuple = audio_features_full_df.iloc[start : start + 10000].values.tolist()
        try:
            query_string = "INSERT INTO PUBLIC.AUDIO_FEATURES_FULL(DANCEABILITY, ENERGY, KEY, LOUDNESS, MODE, SPEECHINESS, ACOUSTICNESS, INSTRUMENTALNESS, LIVENESS, VALENCE, TEMPO, TYPE, ID, URI, TRACK_HREF, ANALYSIS_URL, DURATION_MS, TIME_SIGNATURE, NAME, ARTIST_NAME, RELEASE_DATE) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);"
            cs.executemany(query_string, audio_features_tuple)
        except Exception as e:
            print(e)
            
    ctx.close()
    cs.close()

In [42]:
def checkpoint(*args):
    client = boto3.client(
        's3',
        aws_access_key_id = ID, 
        aws_secret_access_key = SECRET,
        region_name = 'us-west-1'
    )
    
    resource = boto3.resource(
        's3',
        aws_access_key_id = ID, 
        aws_secret_access_key = SECRET,
        region_name = 'us-west-1'
    )

    csv_buffer = StringIO()

    for arg in args:
        try:
            if len(arg) > 0:
                spotify_obj = arg[0][0]
                
                if spotify_obj == 'albums':
                    df = pd.DataFrame([x[1:] for x in arg], columns = ['ind', 'id', 'artist_id']).to_csv(csv_buffer, index=False)

                elif spotify_obj == 'tracks':
                    df = pd.DataFrame([x[1:] for x in arg], columns = ['ind', 'id', 'artist_id', 'album_id']).to_csv(csv_buffer, index=False)

                elif spotify_obj == 'artists':
                    df = pd.DataFrame([x[1:] for x in arg], columns = ['id']).to_csv(csv_buffer, index=False)

                elif spotify_obj == 'playlists':
                    df = pd.DataFrame([x[1:] for x in arg], columns = ['id']) .to_csv(csv_buffer, index=False)
                
                old_obj = [obj['Key'] for obj in client.list_objects(Bucket='spotifyrecsystem')['Contents'] if obj['Key'] == spotify_obj + '/' + spotify_obj + '_data_cache.csv']
                if len(old_obj) > 0:
                    resource.Object('spotifyrecsystem', old_obj[0]).delete()

                resource.Object(bucket, spotify_obj + '/' + spotify_obj + '_data_cache.csv').put(Body=csv_buffer.getvalue())
        except Exception as e:
            raise Exception(e)

In [3]:
ctx = snowflake.connector.connect(
    user = USER,
    password = PASS,
    account = ACCOUNT,
    warehouse = 'COMPUTE_WH',
    database = 'SPOTIFY_DB'
)
cs = ctx.cursor()

sql = "SELECT * FROM PUBLIC.TRACK_IDS"
track_id_list = list(set([x[0] for x in cs.execute(sql)]))


In [30]:
playlist_id_list = []
album_id_list = []
track_id_list = []
artist_id_list = []

In [31]:
for item in cpp.browse('categories')['categories']['items']:
    playlists = cpp.browse('categories', item['id'], 'playlists')
    try:
        for playlist in playlists['playlists']['items']:
            if playlist['id'] not in [x[1] for x in playlist_id_list]:
                playlist_id_list.append(('playlists', playlist['id']))
    except Exception as e:
        print(e)

Found error in response. 

                             URL: https://api.spotify.com/v1/browse/categories/alternative/playlists?limit=50&offset=0& 

                             Message: {'error': {'status': 404, 'message': "Specified id doesn't exist"}} 
 
Found error in response. 

                             URL: https://api.spotify.com/v1/browse/categories/blackhistorymonth/playlists?limit=50&offset=0& 

                             Message: {'error': {'status': 404, 'message': "Specified id doesn't exist"}} 
 
Found error in response. 

                             URL: https://api.spotify.com/v1/browse/categories/throwback/playlists?limit=50&offset=0& 

                             Message: {'error': {'status': 404, 'message': "Specified id doesn't exist"}} 
 
Found error in response. 

                             URL: https://api.spotify.com/v1/browse/categories/sessions/playlists?limit=50&offset=0& 

                             Message: {'error': {'status': 404, 'message': "

In [33]:
checkpoint(playlist_id_list)

In [38]:
ind = 1
for album in cpp.browse('new-releases')['albums']['items']:
    for artist in album['artists']:
        if artist not in [x[1] for x in artist_id_list]:
            artist_id_list.append(('artists', artist['id']))
    if len(album['artists']) > 1:
        for art in range(len(album['artists'])):
            album_id_list.append(('albums', ind, album['id'], album['artists'][art]['id']))
            ind += 1
    else:
        if album['id'] not in [x[2] for x in album_id_list]: 
            album_id_list.append(('albums', ind, album['id'], album['artists'][0]['id']))
            ind += 1

In [43]:
checkpoint(playlist_id_list, artist_id_list, album_id_list)

In [23]:
for y in range(2013, 2021):
    for m in range(1, 13):
        for d in range(1, 32):
            try:
                featured = cpp.browse('featured-playlists', timestamp=datetime(y, m, d, 8))['playlists']['items']
                for playlist in featured:
                    if playlist['id'] not in playlist_id_list: playlist_id_list.append((playlist['id']))
            except Exception as e:
                print(e)
                pass

day is out of range for month
day is out of range for month
day is out of range for month


KeyboardInterrupt: 

In [ ]:
checkpoint(playlist_id_list, artist_id_list, album_id_list)

In [ ]:
if spotify_obj == 'albums':
                    df = pd.DataFrame([x[1:] for x in arg], columns = ['id', 'artist_id']).to_csv(csv_buffer, index=False)

elif spotify_obj == 'tracks':
    df = pd.DataFrame([x[1:] for x in arg], columns = ['id', 'artist_id', 'album_id']).to_csv(csv_buffer, index=False)

elif spotify_obj == 'artists':
    df = pd.DataFrame([x[1:] for x in arg], columns = ['id']).to_csv(csv_buffer, index=False)

else:
    df = pd.DataFrame([x[1:] for x in arg], columns = ['id']) .to_csv(csv_buffer, index=False)

In [24]:
ind = album_id_list[-1][0]
for playlist_id in playlist_id_list:
    try:
        res = cpp.get_playlists(False, '', playlist_id, 'tracks', limit=100)['items']
        for track in res:
            if track['track']['album']['id'] not in [x[2] for x in album_id_list]: 
                if len(track['track']['artists']) > 1:
                    for art in range(len(track['track']['artists'])):
                        album_id_list.append(ind, track['track']['album']['id'], track['track']['artists'][art]['id'])
                        ind += 1
                else:
                    album_id_list.append(ind, track['track']['album']['id'], track['track']['artists'][0]['id'])
                    ind += 1
            for artist in track['track']['artists']:
                if artist['id'] not in artist_id_list: artist_id_list.append(artist['id'])
            if track['track']['id'] not in track_id_list: track_id_list.append(track['track']['id'])
    except Exception as e:
        print('Exception occured: ' + str(e))
        pass

Exception occured: 'NoneType' object is not subscriptable
Exception occured: 'NoneType' object is not subscriptable
Exception occured: 'NoneType' object is not subscriptable
Exception occured: 'NoneType' object is not subscriptable
Exception occured: 'NoneType' object is not subscriptable


In [29]:
checkpoint()

In [30]:
for id in artist_id_list:
    try:
        res = cpp.get_artists(id, 'related-artists')['artists']
        for rel_artist in res:
            if rel_artist['id'] not in artist_id_list: artist_id_list.append(rel_artist['id'])
        if len(artist_id_list) % 100 == 0:
            print(len(artist_id_list))
    except Exception as e:
        print('Exception occured: ' + str(e) + ' ' + id)
        pass

21300
23000
24800
25900
26100
27300
27600
27600
27800
27800
27900
27900
28400
29300
29900
29900
30500
30900
32200
32200
32200
33700
33900
Found error in response. 

                             URL: https://api.spotify.com/v1/artists/5A7d4sfe5ZY1RRf90zlUeo/related-artists?limit=50&offset=0 

                             Code: 401 

                             Reason: Unauthorized 

                             Message: The access token expired 
 
Exception occured: 'artists' 5A7d4sfe5ZY1RRf90zlUeo
34200
34200
34200
34500
35300
35700
36600
36600
37500
37500
37800
37800
37800
37800
38000
38000
38000
38000
38400
38400
38400
38400
38400
39200
39200
39200
39200
39200
39400
40100
40200
41200
41200
42000
42200
42300
42300
42300
42300
Found error in response. 

                             URL: https://api.spotify.com/v1/artists/0zfNkbfXyVpesD3S0XFKB8/related-artists?limit=50&offset=0 

                             Code: 401 

                             Reason: Unauthorized 

              

KeyboardInterrupt: 

In [33]:
checkpoint()

In [ ]:
for id in artist_id_list:
    try:
        res = cpp.get_artists(id, 'albums', include_groups='album,single,appears_on,compilation')
        for album in res['items']:
            if album['id'] not in album_id_list: album_id_list.append(album['id'])
        if len(album_id_list) % 50 == 0:
            print(len(album_id_list))
    except Exception as e:
        print('Exception occured: ' + str(e))
        pass

In [118]:
checkpoint()

In [120]:
for id in album_id_list:
    try:
        res = cpp.get_albums(id, 'tracks')
        for tracks in res['items']:
            if tracks['id'] not in track_id_list:
                track_id_list.append(tracks['id'])
        if len(track_id_list) % 10 == 0:
            print(len(track_id_list))
    except Exception as e:
        print('Exception occured: ' + str(e))
        pass

32110
32200
32200
32200
32250
32270
32270
32270
32270
32270
32290
32370
32370
32380
32380
32380
32380
32380
32480
32650
32690
32690
32700
32700
32740
32800
32800
32800
32830
32920
32920
32920
33170
33320
33320
33350
33460
33460
33460
33480
33480
33480
33480
33530
33530
33530
33530
33530
33530
33530
33540
33540
33840
33840
33840
33910
34000
34000
34000
34000
34010
34010
34060
34060
34060
34060
34060
34060
34060
34060
34060
34060
34060
34060
34060
34060
34060
34100
34100
34100
34100
34100
34100
34100
34100
34100
34100
34100
34100
34110
34110
34250
34250
34340
34340
34340
34340
34340
34350
34370
34390
34390
34400
34400
34400
34410
34410
34410
34410
34490
34560
34560
34690
34710
34710
34710
34710
34710
34710
34710
34710
34710
34710
34710
34710
34710
34710
34710
34710
34710
34710
34710
34710
34710
34710
34710
34710
34710
34710
34710
34710
34710
34710
34810
34810
34810
34810
34810
34860
34880
34880
34880
34880
34880
34880
34880
34880
34920
34920
34920
35020
35020
35040
35170
35170
35200
3520

KeyboardInterrupt: 

In [123]:
checkpoint()

In [23]:
ctx = snowflake.connector.connect(
    user = USER,
    password = PASS,
    account = ACCOUNT,
    warehouse = 'COMPUTE_WH',
    database = 'SPOTIFY_DB'
)
cs = ctx.cursor()

if len(test_af_df) != 0:
    start = (len(test_af_df)//10000 - 1) * 10000
    audio_features_tuple = list(test_af_df.iloc[start : start + 10000].itertuples(index=False, name=None))
    try:
        query_string = "INSERT INTO PUBLIC.AUDIO_FEATURES_ONLY(DANCEABILITY, ENERGY, KEY, LOUDNESS, MODE, SPEECHINESS, ACOUSTICNESS, INSTRUMENTALNESS, LIVENESS, VALENCE, TEMPO, TYPE, ID, URI, TRACK_HREF, ANALYSIS_URL, DURATION_MS, TIME_SIGNATURE) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);"
        cs.executemany(query_string, audio_features_tuple)
    except Exception as e:
        print(e)
        
ctx.close()
cs.close()

False

In [121]:
features_list = []
for dic in features:
    row = []
    for key in dic:
        row.append(dic[key])
    features_list.append(tuple(row))

In [122]:
features_list[0]

(0.455,
 0.0846,
 7,
 -28.236,
 1,
 0.307,
 0.973,
 5e-06,
 0.682,
 0.267,
 127.585,
 'audio_features',
 '52se305TJhSaEFoezbQyqG',
 'spotify:track:52se305TJhSaEFoezbQyqG',
 'https://api.spotify.com/v1/tracks/52se305TJhSaEFoezbQyqG',
 'https://api.spotify.com/v1/audio-analysis/52se305TJhSaEFoezbQyqG',
 254067,
 4)

In [123]:
ctx = snowflake.connector.connect(
    user = USER,
    password = PASS,
    account = ACCOUNT,
    warehouse = 'COMPUTE_WH',
    database = 'SPOTIFY_DB'
)
cs = ctx.cursor()

query_string = "INSERT INTO PUBLIC.AUDIO_FEATURES_ONLY(DANCEABILITY, ENERGY, KEY, LOUDNESS, MODE, SPEECHINESS, ACOUSTICNESS, INSTRUMENTALNESS, LIVENESS, VALENCE, TEMPO, TYPE, ID, URI, TRACK_HREF, ANALYSIS_URL, DURATION_MS, TIME_SIGNATURE) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);"
cs.executemany(query_string, features_list)

TypeError: not all arguments converted during string formatting

In [ ]:
ctx = snowflake.connector.connect(
    user = USER,
    password = PASS,
    account = ACCOUNT,
    warehouse = 'COMPUTE_WH',
    database = 'SPOTIFY_DB'
)
cs = ctx.cursor()

if len(features) != 0:
    start = (len(features)//10000 - 1) * 10000
    try:
        query_string = "INSERT INTO PUBLIC.AUDIO_FEATURES_ONLY(DANCEABILITY, ENERGY, KEY, LOUDNESS, MODE, SPEECHINESS, ACOUSTICNESS, INSTRUMENTALNESS, LIVENESS, VALENCE, TEMPO, TYPE, ID, URI, TRACK_HREF, ANALYSIS_URL, DURATION_MS, TIME_SIGNATURE) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);"
        cs.executemany(query_string, features[start : start + 10000])
    except Exception as e:
        print(e)
        
ctx.close()
cs.close()

In [48]:
features = []
for interval in range(math.ceil(len(track_id_list) / 50)):
    try:
        for dic in [x for x in cpp.get_tracks(track_id_list[interval * 50 : (interval * 50) + 50], 'audio-features')['audio_features'] if x is not None]:
            features.append(dic)
            if len(features) % 10000 == 0:
                audio_features_only_checkpoint()
                print(len(features))
    except Exception as e:
        print('Exception occured: ' + str(e))
        pass

KeyboardInterrupt: 

In [49]:
audio_features_df = pd.DataFrame(features, columns=list(cpp.get_tracks(track_id_list[0], 'audio-features').keys()))

In [ ]:
audio_features_full_df = audio_features_df

for row in range(len(audio_features_df)):
    try:
        track = cpp.get_tracks(audio_features_df.iloc[row]['id'])

        audio_features_full_df.at[row, 'name'] = track['name']
        audio_features_full_df.at[row, 'artist_name'] = track['artists'][0]['name']
        audio_features_full_df.at[row, 'release_date'] = track['album']['release_date']
        
        if len(audio_features_full_df) % 10000 == 0:
            audio_features_full_checkpoint()
            print(len(audio_features_full_df))
    except Exception as e:
        print('Exception occured: ' + str(e))
        pass